In [1]:
# file related
import os
from os.path import join
import datetime

# machine learning
from keras.models import Sequential
from keras.layers import Dense, Input
import tensorflow as tf
from sklearn.model_selection import train_test_split

# utils
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from IPython.display import Audio as play_audio
import scipy as sp
import taunet_utils
import json

# RTNeural special import
import sys
sys.path.append("../RTNeural/python/")
from model_utils import save_model

print(f"TF version {tf.__version__}")

TF version 2.12.0


In [2]:
with open(join("dataset", "saved", "AITD_Dataset_Kristof_beta_1in.json"), "r") as json_file:
    input_data = json.load(json_file)
with open(join("dataset", "saved", "AITD_Dataset_Kristof_beta_1out.json"), "r") as json_file:
    output_data = json.load(json_file)

for d in input_data:
    print(f"{d}: {input_data[d]} -> {output_data[d]}")

1: [0.7748, 0.012, 0.2251, 0.1621, 0.1335, 0.4836, 0.5895, 0.1336, 0.0194, 0.1007, 0.0475, 0.6738, 0.8758] -> [0.7210246, 0.3815809]
2: [0.9566, 0.0322, 0.1695, 0.111, 0.1284, 0.6706, 0.2561, 0.0526, 0.035, 0.0238, 0.0945, 0.7384, 0.2437] -> [0.9999503999999999, 0.1248695]
3: [0.5347, 0.0236, 0.3093, 0.0635, 0.1078, 0.5009, 0.3548, 0.1477, 0.0242, 0.049, 0.0797, 0.2224, 0.9991] -> [0.0452408, 0.6440535333333333]
4: [0.567, 0.0294, 0.312, 0.0813, 0.1557, 0.639, 0.6673, 0.2165, 0.0254, 0.0805, 0.0533, 0.6711, 0.8347] -> [0.2988928, 0.4453225]
5: [0.6793, 0.012, 0.2543, 0.1456, 0.0491, 0.6359, 0.2842, 0.046, 0.0249, 0.055, 0.1009, 0.6149, 0.8181] -> [0.7027057999999999, 0.5654544666666667]
6: [0.543, 0.0087, 0.1529, 0.2627, 0.0063, 0.0794, 0.1626, 0.2719, 0.2605, 0.0991, 0.189, 0.2, 0.9179] -> [0.1832008, 0.8069385333333333]
7: [0.5835, 0.0172, 0.3052, 0.1932, 0.0524, 0.5275, 0.5217, 0.1129, 0.043, 0.0948, 0.0886, 0.6255, 0.4053] -> [0.9999752, 0.07385166666666666]
8: [0.6222, 0.0269, 0.1

In [5]:
def create_model(input_shape):
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(4, activation='relu'))
    # model.add(layers.LSTM(64, return_sequences=True))   # short term memory, useful if input data is related accross vectors
    model.add(Dense(2, kernel_regularizer=tf.keras.regularizers.l2(0.001))) # no activation (linear): continuous mapping of outputs (this is not a classification task!)
    return model

input_shape = (13,)
model = create_model(input_shape)
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 64)                896       
                                                                 
 dense_8 (Dense)             (None, 32)                2080      
                                                                 
 dense_9 (Dense)             (None, 32)                1056      
                                                                 
 dense_10 (Dense)            (None, 16)                528       
                                                                 
 dense_11 (Dense)            (None, 8)                 136       
                                                                 
 dense_12 (Dense)            (None, 4)                 36        
                                                                 
 dense_13 (Dense)            (None, 2)                

In [6]:
combined_data = [(input_data[key], output_data[key]) for key in input_data.keys()]

train_data, test_data = train_test_split(combined_data, test_size=0.05, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

log_dir = "logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

class LogRMSECallback(tf.keras.callbacks.Callback):
    def __init__(self, log_dir):
        super(LogRMSECallback, self).__init__()
        self.log_dir = log_dir

    def on_epoch_end(self, epoch, logs=None):
        mse = logs.get('val_loss')
        rmse = np.sqrt(mse)
        with tf.summary.create_file_writer(self.log_dir).as_default():
            tf.summary.scalar('val_rmse', rmse, step=epoch)

%load_ext tensorboard
%tensorboard --logdir logs/ --port 6006

history = model.fit(
    x=np.array([item[0] for item in train_data]),
    y=np.array([item[1] for item in train_data]),
    validation_data=(
        np.array([item[0] for item in val_data]),
        np.array([item[1] for item in val_data])
    ),
    epochs=50,
    # callbacks=[tensorboard_callback, early_stopping_callback],
    callbacks=[tensorboard_callback, early_stopping_callback, LogRMSECallback(join(log_dir, "rmse"))],
    # batch_size=16
)

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 11700), started 9 days, 21:47:09 ago. (Use '!kill 11700' to kill it.)

Epoch 1/50
2/2 [==============================] - 5s 878ms/step - loss: 0.2589 - accuracy: 0.5682 - val_loss: 0.2084 - val_accuracy: 0.6667
Epoch 2/50
2/2 [==============================] - 0s 375ms/step - loss: 0.2359 - accuracy: 0.5682 - val_loss: 0.1860 - val_accuracy: 0.6667
Epoch 3/50
2/2 [==============================] - 0s 311ms/step - loss: 0.2114 - accuracy: 0.6136 - val_loss: 0.1604 - val_accuracy: 0.6667
Epoch 4/50
2/2 [==============================] - 0s 302ms/step - loss: 0.1878 - accuracy: 0.6136 - val_loss: 0.1362 - val_accuracy: 0.6667
Epoch 5/50
2/2 [==============================] - 0s 340ms/step - loss: 0.1633 - accuracy: 0.6136 - val_loss: 0.1145 - val_accuracy: 0.6667
Epoch 6/50
2/2 [==============================] - 0s 352ms/step - loss: 0.1390 - accuracy: 0.6136 - val_loss: 0.0986 - val_accuracy: 0.6667
Epoch 7/50
2/2 [==============================] - 0s 371ms/step - loss: 0.1283 - accuracy: 0.6136 - val_loss: 0.0932 - val_accuracy: 0.6667
Epoch 8/50
2/2 [====

In [7]:
train_metrics = model.evaluate(
    np.array([item[0] for item in train_data]),
    np.array([item[1] for item in train_data]),
    verbose=0
)
val_metrics = model.evaluate(
    np.array([item[0] for item in val_data]),
    np.array([item[1] for item in val_data]),
    verbose=0
)
test_metrics = model.evaluate(
    np.array([item[0] for item in test_data]),
    np.array([item[1] for item in test_data]),
    verbose=0
)

for item in test_data:
    inp = np.array([item[0]])  # Reshape input data into a batch
    prediction = model.predict(inp)
    print("Input:", item[0])
    print("Expected Output:", item[1])
    print("Predicted Output:", prediction)


print({f"Train {metric_name}": metric_value for metric_name, metric_value in zip(model.metrics_names, train_metrics)})
print({f"Validation {metric_name}": metric_value for metric_name, metric_value in zip(model.metrics_names, val_metrics)})
print({f"Test {metric_name}": metric_value for metric_name, metric_value in zip(model.metrics_names, test_metrics)})
print("RMSE for test set:", np.sqrt(test_metrics[0]))

# save model as .h5, .tflite and .json
model.save(join(log_dir, "taunet.pb"))
tflite_model = tf.lite.TFLiteConverter.from_saved_model(join(log_dir, "taunet.pb")).convert()
with open(join(log_dir, "taunet.tflite"), 'wb') as f:
  f.write(tflite_model)
save_model(model, join(log_dir, "taunet.json"))

1/1 [==============================] - 0s 241ms/step
Input: [0.7748, 0.012, 0.2251, 0.1621, 0.1335, 0.4836, 0.5895, 0.1336, 0.0194, 0.1007, 0.0475, 0.6738, 0.8758]
Expected Output: [0.7210246, 0.3815809]
Predicted Output: [[0.5249382  0.24885817]]
1/1 [==============================] - 0s 51ms/step
Input: [0.543, 0.0087, 0.1529, 0.2627, 0.0063, 0.0794, 0.1626, 0.2719, 0.2605, 0.0991, 0.189, 0.2, 0.9179]
Expected Output: [0.1832008, 0.8069385333333333]
Predicted Output: [[0.3898979  0.20542346]]
1/1 [==============================] - 0s 75ms/step
Input: [0.7945, 0.0326, 0.2023, 0.1039, 0.096, 0.3754, 0.191, 0.1539, 0.0327, 0.0406, 0.0089, 0.79, 0.7839]
Expected Output: [0.8717812, 0.6440764666666666]
Predicted Output: [[0.54984206 0.2568683 ]]
{'Train loss': 0.07281644642353058, 'Train accuracy': 0.6136363744735718}
{'Validation loss': 0.06596104800701141, 'Validation accuracy': 0.6666666865348816}
{'Test loss': 0.12247678637504578, 'Test accuracy': 0.6666666865348816}
RMSE for test set

INFO:tensorflow:Assets written to: logs/20240505-133556\taunet.pb\assets


INFO:tensorflow:Assets written to: logs/20240505-133556\taunet.pb\assets
